## Importar librerias

In [0]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier

## Leer base

In [0]:
ruta_base = "default.base_modelo"

In [0]:
# Leer la tabla usando Spark
ruta_base = "workspace.default.base_modelo"
df_sp = spark.table(ruta_base)
df = df_sp.toPandas()

In [0]:
df.head()

In [0]:
!pip install xgboost
dbutils.library.restartPython()

In [0]:
import xgboost as xgb

## Entrenar hiperparametros - Linea base

In [0]:
# Separar las características y el objetivo
X = df[['periodo_creacion', 'flg_vip', 'tipo_producto_producto_a', 'tipo_producto_producto_b', 'tipo_producto_producto_c', 'monto_1m', 'monto_2m', 'monto_3m', 
        'cantidad_1m', 'cantidad_2m', 'cantidad_3m',
        'ultima_compra_1m', 'crossell']]  # Características
y = df['flg_churn']  # Variable objetivo

# Realizar el train-test split estratificado
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3,  # 30% para el conjunto de prueba
    stratify=y,  # Estratificación por la variable objetivo
    random_state=42  # Semilla para reproducibilidad
)

## Entrenar modelos - Linea base

In [0]:
def entrenar_xgboost_rapido(X_train, X_test, y_train, y_test):
    """
    Entrena un modelo XGBoost para un problema de clasificación binaria utilizando
    parámetros óptimos previamente obtenidos.
    """
    # Parámetros óptimos (reemplaza con los que obtuviste en el GridSearch)
    parametros_optimos = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'use_label_encoder': False,
        'n_estimators': 100,
        'max_depth': 5,
        'learning_rate': 0.1,
        'subsample': 0.8,
        'colsample_bytree': 1,
        'random_state': 42
    }
    
    # Crear y entrenar el modelo
    xgb_model = xgb.XGBClassifier(**parametros_optimos)
    xgb_model.fit(X_train, y_train)
    
    # Predicciones
    y_pred = xgb_model.predict(X_test)
    y_pred_prob = xgb_model.predict_proba(X_test)[:, 1]
    
    # Evaluar el modelo
    print("\nReporte de clasificación:")
    print(classification_report(y_test, y_pred))
    print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_prob):.4f}")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    
    return xgb_model

In [0]:
def entrenar_random_forest_rapido(X_train, X_test, y_train, y_test):
    """
    Entrena un modelo Random Forest para un problema de clasificación binaria utilizando
    parámetros óptimos previamente obtenidos.
    """
    # Parámetros óptimos (reemplaza con los que obtuviste en el GridSearch)
    parametros_optimos = {
        'n_estimators': 100,
        'max_depth': 10,
        'min_samples_split': 5,
        'min_samples_leaf': 2,
        'max_features': 'sqrt',
        'random_state': 42,
        'n_jobs': -1
    }
    
    # Crear y entrenar el modelo
    rf_model = RandomForestClassifier(**parametros_optimos)
    rf_model.fit(X_train, y_train)
    
    # Predicciones
    y_pred = rf_model.predict(X_test)
    y_pred_prob = rf_model.predict_proba(X_test)[:, 1]
    
    # Evaluar el modelo
    print("\nReporte de clasificación:")
    print(classification_report(y_test, y_pred))
    print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_prob):.4f}")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    
    return rf_model

In [0]:
modelo_final_randomforest = entrenar_random_forest_rapido(X_train, X_test, y_train, y_test)

In [0]:
modelo_final = entrenar_xgboost_rapido(X_train, X_test, y_train, y_test)